In [ ]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from helperToolz.mirmazloumi import *
from pympler import asizeof
from other_repos.pyTSEB.pyTSEB import meteo_utils
from other_repos.pyTSEB.pyTSEB import resistances
from other_repos.pyTSEB.pyTSEB import net_radiation
from other_repos.pyTSEB.pyTSEB import clumping_index 
from other_repos.pyTSEB.pyTSEB import TSEB
import rasterio

In [ ]:
# set the year, month and day to estimate evapotranspiration for
year = 2019
month = 'July'
day = 26
comp = 'minVZA'
sharp = 'allpred'

# set output path
tempOut = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/'

# set the parameters from sharpener
mvwin = 15
cv = 0
regrat = 25
s2_masked = True
lst_masked = False

if s2_masked:
    s2Mask = 'S2Masked'
else:
    s2Mask = 'S2notMasked'

if lst_masked:
    lstMask = 'withLSTmask'
else:
    lstMask = 'withoutLSTmask'

# path to era5 raw data
era5_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/'
ssrd_mean_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/ssrd_mean_calc/'

# path_base to sharpenend folder and S2_comp
sharp_pathbase = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened2/{sharp}/ff3f2c872c08977466e5a8dc306d2d2aabc77ad995b0716c30d1c57d0004ebfd/{comp}/{year}/{month}/{day:02d}/{lstMask}/{s2Mask}/Values/'
s2_pathbase = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump2/ff3f2c872c08977466e5a8dc306d2d2aabc77ad995b0716c30d1c57d0004ebfd/'

# sharp_pathbase = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened3/{sharp}/bccec09fdbc7a94b5e2bd6c125d9d1174c2509360a2ce529f48e3512655454b5/{comp}/{year}/{month}/{day:02d}/{lstMask}/{s2Mask}/Values/'
# s2_pathbase = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump3/bccec09fdbc7a94b5e2bd6c125d9d1174c2509360a2ce529f48e3512655454b5/'

# the LST acquisition time should determine which sharpened LST files are associatedto be processed (as they are associated with it)
LST_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/{year}/Daily_AcqTime_{comp}_{year}_{month}.tif' # epsg 4326

# the VZA at the time of LST acquisition is need
VZA_at_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/VZA/comp/{comp}/{year}/Daily_VZA_{comp}_{year}_{month}.tif' # epsg 4326

# the DEM, SLOPE, ASPECT, LAT, LON will be used to sharpen some of the era5 variables (the the resolution of the DEM)
dem_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_FORCE_WARP.tif' # epsg 4326
slope_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/SLOPE_GER_FORCE_WARP.tif' # epsg 4326
aspect_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/ASPECT_GER_FORCE_WARP.tif' # epsg 4326
lat_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LAT_GER_FORCE_WARP.tif' # epsg 4326
lon_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LON_GER_FORCE_WARP.tif' # epsg 4326

# the geopotential is needed for the sharpening as well
geopot_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/geopotential/geopotential_low_res.tif' # epsg 4326

# sharpened LST
LST_file = f'{sharp_pathbase}{comp}_{year}_{month}_{day:02d}_mvwin{mvwin}_cv{cv}_regrat{regrat}_{s2Mask}_{lstMask}.tif' # epsg:3035

# for NDVI calculation (estimating LAI and others) and warping to S2 resolution, we use the S2 composite used for sharpening
# S2_file = [file for file in getFilelist(s2_pathbase, 'vrt', deep=False) if f'HIGHRES_{comp}_{year}_{month}_{day:02d}' in file][0]
S2_file = [file for file in getFilelist(s2_pathbase, 'vrt', deep=True) if 'S2' in file][0]

# find era5 file that matches the month of LST observation
valid_variables = sorted(list(dict.fromkeys(file.split('/')[-2] for file in getFilelist(era5_path, '.grib', deep=True) \
                                   if not any(var in file for var in ['geopotential', 'total_column_water_vapour']))))

# get a list for those era5 files that match the year and month of the provided LST acquisition file
era5_path_list = find_grib_file(getFilelist(era5_path, '.grib', deep=True), LST_acq_file)
era5_path_list = [path for path in era5_path_list if any(variable in path for variable in valid_variables)] # era5 are epsg 4326 and still will be after warping to doy
temp_pressure_checker(era5_path_list)

In [ ]:
# warp datasets needed for calculations to the spatial extent of the sharpened LST
LST_acq_spatial_sub = warp_raster_to_reference(source_path=LST_acq_file, reference_path=S2_file, output_path='MEM', resampling='near')
VZA_at_acq_file_sub = warp_raster_to_reference(source_path=VZA_at_acq_file, reference_path=S2_file, output_path='MEM', resampling='near')
dem_sub = warp_raster_to_reference(source_path=dem_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
slope_sub  = warp_raster_to_reference(source_path=slope_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
aspect_sub = warp_raster_to_reference(source_path=aspect_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
lat_sub = warp_raster_to_reference(source_path=lat_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
lon_sub = warp_raster_to_reference(source_path=lon_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')
geopot_sub = warp_raster_to_reference(source_path=geopot_path, reference_path=S2_file, output_path='MEM', resampling='bilinear')

In [ ]:
# LST_acq_spatial_arr = LST_acq_spatial_sub.GetRasterBand(day).ReadAsArray()
# VZA_at_acq_file_arr = VZA_at_acq_file_sub.GetRasterBand(day).ReadAsArray()
# dem_arr= dem_sub.GetRasterBand(1).ReadAsArray()
# slope_arr = slope_sub.GetRasterBand(1).ReadAsArray()
# aspect_arr = aspect_sub.GetRasterBand(1).ReadAsArray()
# lat_arr = lat_sub.GetRasterBand(1).ReadAsArray()
# lon_arr = lon_sub.GetRasterBand(1).ReadAsArray()
# geopot_arr = geopot_sub.GetRasterBand(1).ReadAsArray()

sufix = 'TESTI2'

# npTOdisk(LST_acq_spatial_arr, S2_file, f'{tempOut}LST_acq_spatial{sufix}.tif', bands = 1, d_type=gdal.GDT_Int64)
# npTOdisk(VZA_at_acq_file_arr, S2_file, f'{tempOut}VZA_at_acq_file_sub{sufix}.tif', bands = 1)
# npTOdisk(dem_arr, S2_file, f'{tempOut}dem_sub{sufix}.tif', bands = 1)
# npTOdisk(slope_arr, S2_file, f'{tempOut}slope_sub{sufix}.tif', bands = 1)
# npTOdisk(aspect_arr, S2_file, f'{tempOut}aspect_sub{sufix}.tif', bands = 1)
# npTOdisk(lat_arr, S2_file, f'{tempOut}lat_sub{sufix}.tif', bands = 1)
# npTOdisk(lon_arr, S2_file, f'{tempOut}lon_sub{sufix}.tif', bands = 1)
# npTOdisk(geopot_arr, S2_file, f'{tempOut}geopot_sub{sufix}.tif', bands = 1)

In [ ]:
# load the era5 variable into cache at LST resolution and read-in the modelled times (one time step per band)
for path in era5_path_list:
    print(f'processing {path}')
    # check if DEM sharpener needs to be applied
    if '100m_u_component_of_wind' in path:
        # do the warping without sharpening
        wind100_u = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)

    elif '100m_v_component_of_wind' in path:
               # do the warping without sharpening
        wind100_v = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)

    # elif 'geopotential' in path:
    #     # do the warping without sharpening
    #     geopot = get_warped_ERA5_at_doy(path_to_era_grib=path, lst_acq_file=LST_acq_file, doy=day)

    elif 'downward' in path: # terrain correction included
        ssrd, szenith, sazimuth, ssrd_nc, ssrd_mean = get_ssrdsc_warped_and_corrected_at_doy(path_to_ssrdsc_grib=path, reference_path=LST_acq_spatial_sub, 
                                                                         lst_acq_file=LST_acq_spatial_sub, doy=day, 
                                                                         slope_path=slope_sub,
                                                                         aspect_path=aspect_sub,
                                                                         dem_path=dem_sub,
                                                                         lat_path=lat_sub,
                                                                         lon_path=lon_sub)

    elif '2m_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        air_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                          lst_acq_file=LST_acq_spatial_sub, doy=day,
                                          sharp_blendheight=100,
                                          sharp_DEM=dem_sub,
                                          sharp_geopot=geopot_sub,
                                          sharp_rate=STANDARD_ADIABAT,
                                          sharpener='adiabatic')

    elif '2m_dewpoint_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        dew_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                          lst_acq_file=LST_acq_spatial_sub, doy=day,
                                          sharp_blendheight=100,
                                          sharp_DEM=dem_sub,
                                          sharp_geopot=geopot_sub,
                                          sharp_rate=MOIST_ADIABAT,
                                          sharpener='adiabatic')

    else: 
        # do warping with DEM sharpening only
        # sanity check
        if not 'surface_pressure' in path:
            raise ValueError('There is and unattended ERA5 variable in the loop - CHECK!!!!')
        else:
            sp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                        lst_acq_file=LST_acq_spatial_sub, doy=day,
                                        sharp_DEM=dem_sub,
                                        sharp_blendheight=100,
                                        sharp_geopot=geopot_sub,
                                        sharp_temp=air_temp,
                                        sharpener='barometric') / 100

In [ ]:
# load the mean ssrd
ds = gdal.Open(f'{ssrd_mean_path}surface_solar_radiation_downward_clear_sky_{year}_{int(MONTH_TO_02D[month])}')
ssrd_mean2 = ds.GetRasterBand(day).ReadAsArray() / 3600

In [ ]:
plotter(ssrd)
plotter(ssrd_mean)
plotter(ssrd_mean2)

In [ ]:
wind_speed_20 = calc_wind_speed(wind100_u, wind100_v) # check wind_u
ssrd_20 = ssrd
# ssrd_mean_20 = warp_np_to_reference(ssrd_mean, f'{ssrd_mean_path}surface_solar_radiation_downward_clear_sky_{year}_{int(MONTH_TO_02D[month])}', LST_file) # check this too!!!!!!
ssrd_mean_20 = ssrd_mean.reshape((1500,1500))

air_temp_20 = air_temp
dew_temp_20 = dew_temp
sp_20 = sp
szenith_20 = szenith
sazimuth_20 = sazimuth

# calculate windspeed

# load vza
vza_ds = VZA_at_acq_file_sub
vza_20 = vza_ds.GetRasterBand(day).ReadAsArray()

# load sharpened LST
lst_ds = gdal.Open(LST_file)
lst_20 =lst_ds.GetRasterBand(1).ReadAsArray()

del wind100_u, wind100_v, ssrd, air_temp, dew_temp, sp, szenith, sazimuth, ssrd_mean, ssrd_nc

In [ ]:
# npTOdisk(ssrd_mean_20, LST_file, f'{tempOut}SSRD_mean2.tif', bands = 1)
# npTOdisk(ssrd_20, LST_file, f'{tempOut}SSRD.tif', bands = 1)
# npTOdisk(air_temp_20, LST_file, f'{tempOut}TEMP.tif', bands = 1)
# npTOdisk(dew_temp_20, LST_file, f'{tempOut}DEW.tif', bands = 1)
# npTOdisk(sp_20, LST_file, f'{tempOut}SP.tif', bands = 1)
# npTOdisk(szenith_20, LST_file, f'{tempOut}ZEN.tif', bands = 1)
# npTOdisk(sazimuth_20, LST_file, f'{tempOut}AZI.tif', bands = 1)
# npTOdisk(wind_speed_20, LST_file, f'{tempOut}WSPEED.tif', bands = 1)
# npTOdisk(lst_20, LST_file, f'{tempOut}LST.tif', bands = 1)
# npTOdisk(vza_20, LST_file, f'{tempOut}VZA.tif', bands = 1)

In [ ]:
condition = (air_temp_20 > 0) & (dew_temp_20 > 0)  & (sp_20 > 0) & (szenith_20 > 0) & (sazimuth_20 > 0) & (wind_speed_20 > 0) & (lst_20 > 0) & (vza_20 > 0)
ssrd_20[~condition] = np.nan
ssrd_mean_20[~condition] = np.nan
air_temp_20[~condition] = np.nan
dew_temp_20[~condition] = np.nan
sp_20[~condition] = np.nan
szenith_20[~condition] = np.nan
sazimuth_20[~condition] = np.nan
wind_speed_20[~condition] = np.nan
# lst_20 = np.ma.masked_where(~condition, lst_20)
# vza_20 = np.ma.masked_where(~condition, vza_20)
lst_20[~condition] = np.nan
vza_20[~condition] = np.nan

In [ ]:
# calculate the NDVI from the S2 composite (following formula from force --> bandnames: (NIR - RED) / (NIR + RED))
S2_ds = gdal.Open(S2_file)
for idx, bname in enumerate(getBandNames(S2_file)):
    if bname == 'RED':
        red = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    elif bname == 'NIR':
        nir = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    else:
        continue
ndvi_20 = (nir - red) / (nir + red)
ndvi_20_ma = np.where(ndvi_20 < 0, np.nan, ndvi_20)
# ndvi_20_ma = np.ma.masked_invalid(ndvi_20)
# ndvi_20_ma = np.ma.masked_where(ndvi_20_ma < 0, ndvi_20_ma)
LAI_np = 0.57*np.exp(2.33*ndvi_20)
LAI_pos = np.where(LAI_np < 0, np.nan, LAI_np)

# estimate canopy height from estimated LAI
hc = hc_from_lai(LAI_pos, hc_max = 20, lai_max = np.nanmax(LAI_np), hc_min=0)

# estimate long wave irradiance
ea = meteo_utils.calc_vapor_pressure(T_K=dew_temp_20)
L_dn = calc_longwave_irradiance(ea = ea, t_a_k = air_temp_20, p = sp_20, z_T = 100, h_C = hc) # ## does that make sense with the 100m!!!!!!!!!!!!!!!!!!!
d_0_0 = resistances.calc_d_0(h_C=hc)
z_0 = resistances.calc_z_0M(h_C=hc)

In [ ]:
# npTOdisk(ndvi_20, LST_file, f'{tempOut}ndvi.tif')
# npTOdisk(ndvi_20_ma, LST_file, f'{tempOut}ndvi_pos.tif')
# npTOdisk(LAI_np, LST_file, f'{tempOut}LAI.tif')
# npTOdisk(LAI_pos, LST_file, f'{tempOut}LAI_pos.tif')
# npTOdisk(hc, LST_file, f'{tempOut}canopy_height.tif')
# npTOdisk(ea, LST_file, f'{tempOut}vapor_pressure.tif')
# npTOdisk(L_dn, LST_file, f'{tempOut}longwave_radiation.tif')
# npTOdisk(d_0_0, LST_file, f'{tempOut}resistanceD.tif')
# npTOdisk(z_0, LST_file, f'{tempOut}resistanceZ.tif')

In [ ]:
# calculate shortwave radiation of soil and canopy
difvis, difnir, fvis, fnir = net_radiation.calc_difuse_ratio(S_dn = ssrd_20, sza = np.nanmean(szenith_20))

skyl = difvis * fvis + difnir * fnir
S_dn_dir = ssrd_20 * (1.0 - skyl)
S_dn_dif = ssrd_20 * skyl

# Leaf spectral properties:{rho_vis_C: visible reflectance, tau_vis_C: visible transmittance, rho_nir_C: NIR reflectance, tau_nir_C: NIR transmittance}
rho_vis_C=np.full(LAI_pos.shape, 0.05, np.float32)
tau_vis_C=np.full(LAI_pos.shape, 0.08, np.float32)
rho_nir_C=np.full(LAI_pos.shape, 0.32, np.float32)
tau_nir_C=np.full(LAI_pos.shape, 0.33, np.float32) 

# Soil spectral properties:{rho_vis_S: visible reflectance, rho_nir_S: NIR reflectance}
rho_vis_S=np.full(LAI_pos.shape, 0.07, np.float32)
rho_nir_S=np.full(LAI_pos.shape, 0.25, np.float32)

# F = local LAI
F = LAI_pos / 1
# calculate clumping index
Omega0 = clumping_index.calc_omega0_Kustas(LAI = LAI_np, f_C = 1, x_LAD=1)
Omega = clumping_index.calc_omega_Kustas(Omega0, np.nanmean(szenith_20))
LAI_eff = F * Omega

Sn_C, Sn_S = net_radiation.calc_Sn_Campbell(lai = LAI_pos, sza = np.mean(szenith_20), S_dn_dir = S_dn_dir, S_dn_dif = S_dn_dif, fvis = fvis,
                                    fnir = fnir, rho_leaf_vis = rho_vis_C, tau_leaf_vis = tau_vis_C, rho_leaf_nir = rho_nir_C, 
                                    tau_leaf_nir = tau_nir_C, rsoilv = rho_vis_S, rsoiln = rho_nir_S, x_LAD=1, LAI_eff=LAI_eff)

# calculate other roughness stuff
z_0M, d = resistances.calc_roughness(LAI=np.nanmean(LAI_pos), h_C=hc, w_C=1, landcover=11, f_c=None)
fg = calc_fg_gutman(ndvi = ndvi_20_ma, ndvi_min = np.nanmin(ndvi_20), ndvi_max = np.nanmax(ndvi_20))

In [ ]:
emis_C = 0.98
emis_S = 0.95
h_C = hc 
z_u = 100
z_T = 100

In [ ]:
output = TSEB.TSEB_PT(lst_20, vza_20, air_temp_20, wind_speed_20, ea, sp_20, Sn_C, Sn_S, L_dn, LAI_pos, h_C, emis_C, emis_S, 
                      z_0M, d, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None,
                      kB=0.0, massman_profile=None, verbose=True)

ld = output[6]/ssrd_20
heat_latent_scaled = ssrd_mean_20 * ld
et_daily_p = TSEB.met.flux_2_evaporation(output[6], t_k=air_temp_20, time_domain=24)

In [ ]:
plotter(et_daily_p)

In [ ]:
out1 = output[4]
#out1[out1<100] = np.nan
plotter(out1)

In [ ]:
plotter(fg)

In [ ]:
storPath = f'{tempOut}{comp}_{year}_{month}_{day}_mvwin{mvwin}_cv{cv}_regrat{regrat}_{lst_masked}_{s2_masked}_{sharp}_ET.tif'
npTOdisk(et_daily_p, LST_file, storPath)

In [ ]:
!jupyter nbconvert --to script estimate_Evapotrans.ipynb --output /home/potzschf/repos/evapo_scripts/Guzinski/py/estimate_Evapotrans_04_09

In [ ]:

Tile_X = [67]
Tile_Y = [42]


tiles_to_process = getUniqueIDfromTILESXY(Tile_X, Tile_Y)



In [ ]:
qq = 0
year = 2019
month = 'July'
for day in range(19, 28, 1):
    for comp in ['maxLST', 'minVZA']:
        for sharp in ['allpred', 'S2only']:
            for s2Mask in ['S2Masked', 'S2notMasked']:
                for lstMask in ['withLSTmask', 'withoutLSTmask']:
                    for tileID, uniqueFolder in zip(['X64_Y42', 'X67_Y42'], ['ff3f2c872c08977466e5a8dc306d2d2aabc77ad995b0716c30d1c57d0004ebfd', 'bccec09fdbc7a94b5e2bd6c125d9d1174c2509360a2ce529f48e3512655454b5']):
                        # path_base to sharpenend folder and S2_comp
                        qq += 1
                        print(qq)

In [ ]:
predi = 'allpred'
rand_foldname = 'TEST3X3'
temp_dump = '/data/Aldhani/eoagritwin/et/Sentinel3//LST/LST_values/tempDump3X3/'
temp_dump_fold = f'{temp_dump}{rand_foldname}/'
highResFilename = f'{temp_dump_fold}HGIHRES_maxLST_2019_JUly_27_watermask_S2_agromask.tif'
outFolder = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened_3X3/{predi}/{rand_foldname}/'

print(f'{outFolder}{'/'.join(highResFilename.split('.')[0].split('_')[2:6])}/')

# for idx, highResFilename in enumerate(highRes_files):
#     lowResFilename = lowRes_files[idx]
#     f1 = f'{outFolder}{'/'.join(highResFilename.split('.')[0].split('_')[2:6])}/'
#     for maskname, mask_lowRes in zip(['withoutLSTmask'], [lowmask_bin_path]):
#     # for maskname, mask_lowRes in zip(['withoutLSTmask', 'withLSTmask'], ['', lowmask_bin_path]):
#         lowResMaskFilename = mask_lowRes
#         f2 = f'{f1}{maskname}/'
#         for movWin in [15, 30]:
#             for cv in [0, 10]:
#                 for regrat in [0.25, 0.5]:
#                     kombi = f'mvwin{movWin}_cv{cv}_regrat{int(regrat*100):02d}_{highRes_names[idx]}_{maskname}'
#                     f3 = f'{f2}{highRes_names[idx]}/'
#                     os.makedirs(f'{f3}Residuals/', exist_ok=True)
#                     os.makedirs(f'{f3}Values/', exist_ok=True)
#                     joblist.append([highResFilename, 
#                                 lowResFilename,
#                                 lowResMaskFilename,
#                                 cv, movWin, regrat,
#                                 f'{f3}{'_'.join(highResFilename.split('.')[0].split('_')[2:6])}_{kombi}.tif'])
